In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
def filterBike(pid,records):
    if pid ==0:
        next(records)
    for record in records:
        fields =  record.split(',')
        if(fields[6]=='Greenwich Ave & 8 Ave' and fields[3].startswith('2015-02-01')):
            yield(fields[3][:19],1)


In [4]:
bike = sc.textFile('citibike.csv')
bike.take(1)[0].split(',')


['cartodb_id',
 'the_geom',
 'tripduration',
 'starttime',
 'stoptime',
 'start_station_id',
 'start_station_name',
 'start_station_latitude',
 'start_station_longitude',
 'end_station_id',
 'end_station_name',
 'end_station_latitude',
 'end_station_longitude',
 'bikeid',
 'usertype',
 'birth_year',
 'gender']

In [5]:
matchedBike = bike.mapPartitionsWithIndex(filterBike)
matchedBike.take(2)

[('2015-02-01 00:05:00', 1), ('2015-02-01 00:05:00', 1)]

In [6]:
def filterTaxi(pid,lines):
    if pid == 0:
        next(lines)
    import pyproj
    proj = pyproj.Proj(init = 'epsg:2263',preserve_units = True)
    station = proj(-74.00263761,40.73901691)
    print(station.)
    radius = 1320**2
    for trip in lines:
        fields  = trip.split(',')
        if 'NULL' in fields[4:6]: continue
        dropoff  = proj(fields[5],fields[4])
        distance = ((station[0]-dropoff[0])**2 + (station[1] - dropoff[1])**2)
        if distance<=radius:
            yield(fields[1][:19],0)

In [7]:
# def filterTaxi(pid, lines):
#    if pid ==0:
#        next(lines)
#    import pyproj
#    proj = pyproj.Proj(init='epsg:2263', preserve_units=True)
#    station = proj(-74.00263761, 40.73901691)
#    radius = 1320**2
#    for trip in lines:
#        fields = trip.split(',')
#        if 'NULL' in fields[4:6]: continue
#        dropoff = proj(fields[5], fields[4])
#        distance = ((station[0] - dropoff[0])**2 + (station[1] - dropoff[1])**2)
#        if distance <= radius:
#            yield (fields[1][:19],0)

# taxi = sc.textFile('yellow.csv')
# taxi.take(2)[1].split(',')
# matchedTaxi = taxi.mapPartitionsWithIndex(filterTaxi)
# matchedTaxi.take(2)

In [8]:
taxi = sc.textFile('yellow.csv')
taxi.take(2)[1].split(',')
matchedTaxi = taxi.mapPartitionsWithIndex(filterTaxi)
matchedTaxi.take(2)

[('2015-02-01 00:11:03', 0), ('2015-02-01 00:10:23', 0)]

In [9]:
allTrips = (matchedBike+matchedTaxi).sortByKey().cache()
allTrips.take(7)

[('2015-02-01 00:03:12', 0),
 ('2015-02-01 00:04:39', 0),
 ('2015-02-01 00:05:00', 1),
 ('2015-02-01 00:05:00', 1),
 ('2015-02-01 00:05:38', 0),
 ('2015-02-01 00:06:15', 0),
 ('2015-02-01 00:07:07', 0)]

In [10]:
def connectTrips(_, records):
   import datetime
   lastTaxiTime = None
   count = 0
   for dt, mode in records:
       t = datetime.datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')
       if mode == 1:
           if lastTaxiTime != None:
               diff = (t-lastTaxiTime).total_seconds()
               if  0 <= diff <= 10:
                   count += 1
       else:
           lastTaxiTime = t
   yield count


allTrips.mapPartitionsWithIndex(connectTrips).reduce(lambda x,y: x+y)

42